## 稀疏矩阵之 COO, CSR, CSC

本文讨论 scipy 中的稀疏向量，因为前几天在使用 `pandas.get_dummies` 时遇到了一些问题：

```python
X = pd.get_dummies(dataset, columns=dataset.columns, sparse=True).to_coo()

X_train = X[:train.shape[0]]
X_test = X[train.shape[0]:]
```

上面的代码会报错，因为 COO 格式的稀疏矩阵不能进行切片。需要下面这样才行：

```python
X = X.tocsr()
X_train = X[:train.shape[0]]
X_test = X[train.shape[0]:]
```

今天花了点时间，了解了下稀疏矩阵常见的这三种格式。

### COO - Coordinate Format

COO 格式通过坐标来定义稀疏矩阵，这时最符合直觉的一种格式，只需要指定非零元素的行、列、具体数值即可。下面是个例子：

In [1]:
import numpy as np
from scipy.sparse import coo_matrix

row  = np.array([0, 1, 2])
col  = np.array([2, 1, 0])
data = np.array([1, 2, 3])

print(coo_matrix((data, (row, col)), shape=(3, 3)))

  (0, 2)	1
  (1, 1)	2
  (2, 0)	3


可以使用 `toarray()` 或者 `todense` 转为稠密数组或者矩阵。

In [2]:
coo_matrix((data, (row, col)), shape=(3, 3)).toarray()

array([[0, 0, 1],
       [0, 2, 0],
       [3, 0, 0]])

如果在同一个位置上指定了多个数，那么在转为稠密矩阵或者转为其他格式的时候，相同位置额值会累加起来：

In [3]:
row  = np.array([0, 1, 2, 0])
col  = np.array([2, 1, 0, 2])
data = np.array([1, 2, 3, 9])

A = coo_matrix((data, (row, col)), shape=(3, 3))
A.toarray()

array([[ 0,  0, 10],
       [ 0,  2,  0],
       [ 3,  0,  0]])

COO 格式便于手动构造稀疏矩阵，但是它存在诸多短板，没办法做切片。

In [4]:
A[:1]

TypeError: 'coo_matrix' object is not subscriptable

为了支持行和列的切片，需要使用 CSR 和 CSC 格式。COO 格式可以使用 `tocsr` 和 `tocsc` 来转换格式。

In [5]:
A.tocsr()

<3x3 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

### CSR - Compressed Sparse Row

使用稀疏矩阵其中一个动机就是节省存储空间，CSR 格式在存储的时候是按行压缩存储空间，COO 格式中，每个元素都需要三个值（行+列+值）来描述，在 CSR 和后面提到的 CSC 中能够使用更少的信息来表示稀疏矩阵。

可以使用 和 COO 相同的方式来创建 CSR 和 CSC 格式的系数矩阵。行+列+值，就可以了。

In [6]:
from scipy.sparse import csr_matrix

row  = np.array([0, 0, 1, 2, 2, 2])
col  = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
csr_matrix((data, (row, col)), shape=(3, 3)).toarray()

array([[1, 0, 2],
       [0, 0, 3],
       [4, 5, 6]], dtype=int64)

在 CSR 中还有另一种创建方式，其参数形式如下：

```python
csr_matrix((data, indices, indptr), [shape=(M, N)])
```

下面举例说明：

In [7]:
indptr = np.array([0, 2, 3, 6])
indices = np.array([0, 1, 2, 0, 1, 2])
data    = np.array([1, 2, 3, 4, 5, 6])
A = csr_matrix((data, indices, indptr), shape=(3, 3))
A.toarray()

array([[1, 2, 0],
       [0, 0, 3],
       [4, 5, 6]])

这是一种新的创建 CSR 矩阵的形式，在 CSR 格式中，数据是按行存储在一个列表中的，下面举例说明各个参数的含义：

`indptr` 中包含 4 个元素，这 4 个元素可以确定 3 个区间，`0-2, 2-3, 3-6`。以 `0-2` 为例：

```python
indices[0:2] -> [0, 1]
```

这说明，在第 1 行中 `0, 1` 两列有值。

```python
data[0:2] -> [1, 2]
```

这说明第一行中的值是 `1, 2`。

有了 indices 指示的列信息，以及 data 中指示的数值信息，稀疏矩阵就能确定了。

同理，`2-3` 区间就用于确定第 2 行的列和数值信息：

```python
indices[2:3] -> [2] # 列号
data[2:3] -> [3] # 值
```

说明：`A[1, [2]] = [3]`

CSR 可以支持切片，在做行切片的时候速度很快，这得益于其存储格式。在做列切片的时候就速度就较慢了。

In [8]:
row = np.random.randint(0, 2000, 50000)
col = np.random.randint(0, 2000, 50000)
data = np.ones(50000)
A = csr_matrix((data, (row, col)))
%timeit _ = A[:1000,:]
%timeit _ = A[:,:5000]

311 µs ± 39.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
428 µs ± 71.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### CSC - CCompressed Sparse Column

CSC 格式和 CSR 格式类似，只是在压缩的时候是按列压缩。CSC 格式在做列切片的时候会比较快。

In [9]:
from scipy.sparse import csc_matrix

row  = np.array([0, 2, 2, 0, 1, 2])
col  = np.array([1, 0, 1, 2, 2, 2])
data = np.array([1, 2, 3, 4, 5, 6])
csc_matrix((data, (row, col)), shape=(3, 3)).toarray()

array([[0, 1, 4],
       [0, 0, 5],
       [2, 3, 6]], dtype=int64)

In [10]:
indptr = np.array([0, 2, 3, 6])
indices = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
csc_matrix((data, indices, indptr), shape=(3, 3)).toarray()

array([[1, 0, 4],
       [0, 0, 5],
       [2, 3, 6]])